# Processing surfaces and points in napari - from python
[napari-process-points-and-surfaces]() (nppas) allows post-processing of surface and point cloud data. The available functionality uses [open3d](http://www.open3d.org/) in the background.
For detailed explanation of the underlying algorithms, please refer to the [open3d documentation](http://www.open3d.org/docs/release/).

In [1]:
import napari
from napari.utils import nbscreenshot
import numpy as np

from napari_process_points_and_surfaces import convex_hull,\
            filter_smooth_simple,\
            filter_smooth_laplacian,\
            filter_smooth_taubin,\
            simplify_vertex_clustering,\
            simplify_quadric_decimation,\
            subdivide_loop,\
            labels_to_centroids,\
            sample_points_uniformly,\
            sample_points_poisson_disk,\
            voxel_down_sample,\
            points_to_labels,\
            points_to_convex_hull_surface,\
            surface_from_point_cloud_alpha_shape,\
            surface_from_point_cloud_ball_pivoting,\
            label_to_surface,\
            largest_label_to_surface,\
            fill_holes

In [ ]:
viewer = napari.Viewer(ndisplay=3)

def hide_all(viewer):
    for layer in viewer.layers:
        layer.visible = False
        
def show_all(viewer):
    for layer in viewer.layers:
        layer.visible = True

INFO:numexpr.utils:NumExpr defaulting to 4 threads.
INFO:xmlschema:Resource 'XMLSchema.xsd' is already loaded


In [ ]:
from skimage.data import cells3d
nuclei = cells3d()[:,1,60:120,30:80]
nuclei.shape

In [ ]:
viewer.add_image(nuclei)

nbscreenshot(viewer)

In [ ]:
from skimage.measure import label
labels = label(nuclei>20000)

viewer.add_labels(labels)

nbscreenshot(viewer)

In [ ]:
surface = largest_label_to_surface(labels)

surface_layer = viewer.add_surface(surface, blending='translucent', shading='flat')
surface = surface_layer.data

nbscreenshot(viewer)

## Surface filtering

In [ ]:
hide_all(viewer)

viewer.add_surface(convex_hull(surface))

nbscreenshot(viewer)

In [ ]:
hide_all(viewer)

viewer.add_surface(filter_smooth_simple(surface, 10))

nbscreenshot(viewer)

In [ ]:
hide_all(viewer)

viewer.add_surface(filter_smooth_laplacian(surface, 1))

nbscreenshot(viewer)

In [ ]:
hide_all(viewer)

viewer.add_surface(filter_smooth_taubin(surface, 100))

nbscreenshot(viewer)

## Surface simplification

In [ ]:
hide_all(viewer)

viewer.add_surface(simplify_vertex_clustering(surface))

nbscreenshot(viewer)

In [ ]:
hide_all(viewer)

viewer.add_surface(simplify_quadric_decimation(surface))

nbscreenshot(viewer)

In [ ]:
hide_all(viewer)

viewer.add_surface(subdivide_loop(surface))

nbscreenshot(viewer)

## Point clouds

In [ ]:
hide_all(viewer)

labels_to_centroids(labels, viewer=viewer)
centroids = viewer.layers[-1].data

nbscreenshot(viewer)

In [ ]:
hide_all(viewer)
    
point_cloud = sample_points_uniformly(surface, number_of_points=1000, viewer=viewer)

nbscreenshot(viewer)

In [ ]:
hide_all(viewer)

sample_points_poisson_disk(surface, number_of_points=1000, viewer=viewer)
points_layer = viewer.layers[-1]

nbscreenshot(viewer)

In [ ]:
hide_all(viewer)
    
voxel_down_sample(points_layer.data, viewer=viewer)

nbscreenshot(viewer)

## From points to labels

In [ ]:
hide_all(viewer)

viewer.add_labels(points_to_labels(points_layer.data, labels))

nbscreenshot(viewer)

## From points to surfaces

In [ ]:
hide_all(viewer)

surface_from_points = surface_from_point_cloud_alpha_shape(points_layer.data)
viewer.add_surface(surface_from_points)

nbscreenshot(viewer)

In [ ]:
hide_all(viewer)

surface_from_points = surface_from_point_cloud_ball_pivoting(points_layer.data)
viewer.add_surface(surface_from_points)

nbscreenshot(viewer)

In [ ]:
hide_all(viewer)

viewer.add_surface(points_to_convex_hull_surface(points_layer.data))

nbscreenshot(viewer)

In [ ]:
hide_all(viewer)

surface = points_to_convex_hull_surface(points_layer.data)
faces_del = np.delete(surface[1], np.random.randint(0, surface[1].shape[0], 10), axis=0)
viewer.add_surface((surface[0], faces_del), name='Surface with hole')
nbscreenshot(viewer)

In [ ]:
hide_all(viewer)
viewer.add_surface(fill_holes((surface[0], faces_del), size_limit=1000), name='Filled hole')
nbscreenshot(viewer)

In [ ]:
show_all(viewer)

viewer.grid.enabled = True
viewer.grid.stride = -1

nbscreenshot(viewer)